In [1]:
import requests
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv(r"C:\Users\huang\Downloads\tmassignment\tm_docid.csv")

C:\Users\huang\AppData\Local\Temp\ipykernel_25032\2578986013.py:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\huang\Downloads\tmassignment\tm_docid.csv")


In [3]:
df = df.drop(['rf_id', 'reg_no', 'intl_reg_no', 'count'], axis=1)

In [4]:
df.head(5)

,serial
0,71231446
1,71538408
2,71510081
3,71224781
4,71230951


In [5]:
def fetchHtml(serial):
    res = requests.get(f"https://tsdr.uspto.gov/maintenanceview/sn{serial}")
    if res.status_code==200:
        return res.text
    else:
        return 'Error'

In [6]:
def getImage(serial):
    res = requests.get(f"https://tsdr.uspto.gov/img/{serial}/large?1690917097424")
    if res.status_code==200:
        with open(f"./images/{serial}.png", "wb") as f:
            for chunk in res:
                f.write(chunk)

In [7]:
def appToFile(name, info):
    with open(name, "a") as f:
        f.write(info)

In [9]:
skips = []

f = open(f"./else.txt", "r")
for line in f:
    skips.append(int(line[:7]))

ValueError: invalid literal for int() with base 10: '93341 i'

In [10]:
for serial_num in tqdm(df.values.tolist()):
    if serial_num[0] in skips:
        continue
    else:
        try:
            html = fetchHtml(serial_num[0])
            if html.find("Renewal Granted")!=-1:
                try:
                    getImage(serial_num[0])
                    appToFile("./success.txt", f"{serial_num[0]} is renewed and image is downloaded\n")
                except:
                    appToFile("./error.txt", f"{serial_num[0]} is renewed but image didn't download\n")
            else:
                appToFile("./else.txt", f"{serial_num[0]} is no longer alive\n")
        except:
            appToFile("./error.txt", f"{serial_num[0]} fetch didn't request\n")

  0%|          | 74/6764462 [00:13<289:21:12,  6.49it/s]